Se cargan las líbrerias

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
import sklearn.neural_network




Se cargan los datos

In [0]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/sms-spam.csv",
    sep = ',',
    thousands = None,
    decimal = '.',
    encoding='latin-1')
data

Creamos una copia de los datos (mails) para operar sobre ella y luego eliminamos los afijos morfológicos de las palabras, dejando únicamente su raíz; ésto lo agregamos a una nueva columna.

In [52]:
mails=data.copy()
stemmer = PorterStemmer()

mails['stemmed'] = mails.text.apply(lambda x: ' '.join([stemmer.stem(w) for w in x.split() ]))

mails.head(10)



,type,text,stemmed
0,ham,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. avail onli in b..."
1,ham,Ok lar... Joking wif u oni...,Ok lar... joke wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entri in 2 a wkli comp to win FA cup fina...
3,ham,U dun say so early hor... U c already then say...,U dun say so earli hor... U c alreadi then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah I don't think he goe to usf, he live aroun..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darl it' been 3 week' now an...
6,ham,Even my brother is not like to speak with me. ...,even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...,As per your request 'mell mell (oru minnaminun...
8,spam,WINNER!! As a valued network customer you have...,winner!! As a valu network custom you have bee...
9,spam,Had your mobile 11 months or more? U R entitle...,had your mobil 11 month or more? U R entitl to...


Hallamos la matriz de palabras del documento, ignorando aquellas palabras que tengan poca frecuencia (menores a 5) y convertimos todo a minuscula 

In [53]:
##
## Matriz de términos del documento
##
count_vect = CountVectorizer(
    analyzer='word',        # a nivel de palabra
    lowercase=True,         # convierte a minúsculas
    stop_words='english',   # stop_words en inglés
    min_df=5)               # ignora palabras con baja freq

##
## Aplica la función al texto
##
df_dtm = count_vect.fit_transform(mails.stemmed)

##
## Las filas contienen los mensajes
## y las clomunas los términos
##
df_dtm.shape

(5574, 1540)

Dividimos nuestros datos en entrenamiento y test, donde x es la matriz de terminos, y Y es la columna "type" de mails.
Luego aplicamos regresión logistica con cross-validation con LogisticRegressionCV quien aplica Kfold estratificado por default



In [0]:
X = df_dtm.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, mails["type"], test_size=0.25)

model = LogisticRegressionCV(solver='liblinear', penalty='l1')
model.fit(X_train, y_train)
pred = model.predict(X_test)


Ahora, podemos ver que porcentaje de palabras es capaz de acertar nuestro modelo

In [55]:
#accuracy_score(y_test,pred)
model.score(X_train,y_train)

0.99688995215311

Ahora, hagamoslo bajo un modelo de red neuronal con MLPregressor.

In [0]:
y=mails["type"].copy()

y=y.replace("ham",0) 
y=y.replace("spam",1)

XX_train, XX_test, yy_train, yy_test = train_test_split(X, y, test_size=0.25)

m = sklearn.neural_network.MLPRegressor(
                 hidden_layer_sizes = (2,1),  # Una capa oculta con 2 neurona
                 activation = 'logistic',    #  {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
                 solver = 'sgd',             #  {‘lbfgs’, ‘sgd’, ‘adam’}
                 alpha = 0.0,                #
                 learning_rate_init = 0.1,   # Valor de la tasa de aprendizaje
                 learning_rate = 'constant', # La tasa no se adapta automáticamente
                 verbose = False,            # Reporte del proceso de optimización
                 shuffle = True,             #
                 tol = 1e-8,                 #
                 max_iter = 25000,           # Número máximo de iteraciones
                 momentum = 0.0,             #
                 nesterovs_momentum = False) #


In [59]:
m.fit(XX_train,yy_train)                                  # Entrena el modelo

MLPRegressor(activation='logistic', alpha=0.0, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(2, 1), learning_rate='constant',
             learning_rate_init=0.1, max_fun=15000, max_iter=25000,
             momentum=0.0, n_iter_no_change=10, nesterovs_momentum=False,
             power_t=0.5, random_state=None, shuffle=True, solver='sgd',
             tol=1e-08, validation_fraction=0.1, verbose=False,
             warm_start=False)

Y ahora, veamos que tan bueno es el modelo

In [61]:
m.score(XX_test,yy_test)



0.851645417108919